### 3.2. Создайте функцию, которая принимает две даты date_start, date_end (по аналогии с первой функцией), затем строит и выводит корреляционную матрицу продаж (amount из таблицы revenue) за заданный промежуток времени. В данном случае условные "случайные величины" - это временные ряды продаж (amount) разных товаров.

In [1]:
def my_func32(date_start, date_end):
    import psycopg2
    from collections import defaultdict
    import numpy as np
    import pandas as pd
    import datetime

    #Подключаемся к БД:
    conn = psycopg2.connect(dbname='AdHacks', user='postgres',
                            password='111222', host='localhost', port="5433")
    cursor = conn.cursor()

    # Составляем запрос:
    SQL1 = "SELECT DISTINCT product, SUM(revenue) AS revenue, date FROM revenue WHERE date BETWEEN '"
    SQL2 = "' AND '"
    SQL3 = "' GROUP BY date, product ORDER BY date;"
    SQL = SQL1 + date_start + SQL2 + date_end + SQL3
    cursor.execute(SQL)
    records = cursor.fetchall()
    cursor.close()
    conn.close()

    #Формируем упорядоченный набор попавших дат за указанный период:
    date_set = sorted(list({records[n][2] for n in range(len(records))}))
    # Формируем набор товаров:
    keys_set = (list({records[n][0] for n in range(len(records))}))

    # Составляем словарь, где отражено, в какие даты была получена выручка, включая и нулевую:
    dict1 = defaultdict(list)
    for m, n, k in records:
        dict1[k].append([m, n])

    for item in date_set:
        if len(dict1[item]) != len(keys_set):
            for item2 in list(set(keys_set) - set(dict1[item][n][0] for n in range(len(dict1[item])))):
                dict1[item].append([item2, 0])

    # Делаем список значений словаря одномерным:
    data_list = ([list(dict1.values())[n][j] for n in range(len(dict1.values())) for j in range(len(keys_set))])


    # Строим финальный словарь с правильной последовательностью продаж:
    dict_final = defaultdict(list)
    for a, b in data_list:
        dict_final[a].append(b)
    # Рассчитываем корреляционную матрицу:
    array_results = (np.corrcoef(list(dict_final.values())))

    # Составляем таблицу из корреляционной матрицы:
    null_values = np.zeros((len(array_results), 1))
    corr_matrix = pd.DataFrame(np.hstack((null_values, array_results)), columns=["Товар"] + sorted([n for n in keys_set]))
    corr_matrix.loc[:,"Товар"] = sorted([n for n in keys_set])
    corr_matrix.set_index("Товар", inplace=True)

    return corr_matrix

In [3]:
print(my_func32('2000/01/01', '2015-06-01'))

              A         B         C         D         E         G         H  \
Товар                                                                         
A      1.000000  0.083235  0.062590  0.049714  0.100810  0.006385 -0.007290   
B      0.083235  1.000000  0.028666  0.129882  0.033634  0.051690  0.058881   
C      0.062590  0.028666  1.000000  0.015930  0.017488  0.055275  0.022671   
D      0.049714  0.129882  0.015930  1.000000  0.053530  0.018822 -0.019953   
E      0.100810  0.033634  0.017488  0.053530  1.000000 -0.012228  0.032403   
G      0.006385  0.051690  0.055275  0.018822 -0.012228  1.000000  0.088740   
H     -0.007290  0.058881  0.022671 -0.019953  0.032403  0.088740  1.000000   
K      0.087204  0.019208  0.007818 -0.023457  0.052227  0.063599  0.019581   
M      0.046329  0.104863 -0.007393  0.048593 -0.019578  0.008142 -0.009499   
N      0.053726  0.034884 -0.036194 -0.008903  0.011300  0.011222  0.027987   
X     -0.014294  0.025927  0.054787 -0.016605 -0.018